# Thesis Review
Sept, 25, 2024:

### Maximum Torque per Ampere (MTPA) and Maximum Torque per Voltage (MTPV) BLDC Schemes

BLDC motor drive consists of:
- Permanent magnet synchronous machine (PMSM)
- Voltage-source inverter (VSI) (DC-AC converter)


Two Control Schemes:
- 120 degree (each transistor conducts for 120 degrees per elec cycle, for two 60-deg segmenets, phase current is 0) -> better for achieving MTPV
- 180 degree (each transistor conducts for 180 degrees per elec cycle, phases go between positive and negative voltage) -> high power delivery

Phase voltage is advanced relative to rotor poles (advance in firing angle $\phi_v$). Implemented by shifting conduction logics w.r.t rotor position (i.e. offsetting hall effect sensors)

Three Schemes to control firing angle:
- Common operating mode (COM): Fixed firing angle ($\phi_v = 0$ for 180deg control scheme, $\phi_v = 30$ deg for 120deg control scheme)
- Maximum Torque per Ampere (MTPA): Maximizing torque per stator current ratio -> better for steady state operation
- Maximum Torque per Voltage (MTPV): Maximizing torque per dc source voltage -> faster speed response than MTPA

Optimization:
- To have the benefits of both MTPA and MTPV, the controller should allow MTPA for low-speed range and MTPV at high speed. 
- 120 deg commutation has non-complementary switching (this means that for 60 degrees, the two transistors are not opposite values. i.e. when the phase current is off, both transistors are off). Stator inductance causes delay in changing the phase current to zero, goes to zero at time depending on machines time constant and operating conditions. 
- For MTPA, we want $i_d$ to be zero (only quadrature axis current contributes to torque). For hall-effect BLDC, can use a PI controller to fix current to zero -> tradeoff is increased complexity with high-order filter -> need computationally efficient method
- MTPV. This is implemented either with analytical approximations or lookup tables (LUT). Values are tabulated from simulations or experiments. LUTs remove computational burden on microprocessors
- Switching methods other than 120deg and 180deg. (i.e. 150, 160, 165 degree)

Objectives:
- Achieve MTPA and MTPV with different switching logics. Make a hybrid control scheme for 180 deg operation
- Reduce computational cost for 120 deg operation
- Extend switching logic to angles between 120 and 180 deg. 

Test setup:
- Voltage-source inverter board controlled by microcontroller.
- Inverter supplied by DC source
- Dynamometer (device that measured torque, force, or power) as mech load 
- Computer for data aquisition 

# Objective 1: Hybrid model with 180 deg switching logic:
- MTPV during transients
- MTPA during steady-state operation


# Modeling PMSM:

### Phase Coordinates:

State voltage equations in phase coordinates:
- $V_{abcs} = r_si_{abcs} + \frac{d\lambda_{abcs}}{dt}$

Flux linkage equations:
- $\lambda_{abcs} = L_si_{abcs} + \lambda_m'$

Inductance matrix:
- Self-inductance: $L_{ls} + L_m$ (combination of leakage and magnetizing)
- Mutual-inductance: $-0.5*L_{m}$ (since phases are offset by 120 deg, cos(120) = -0.5) 

Rotor PM Magnet Flux Linkage:
- $\lambda_m' = \lambda_m'*[sin(\theta_r), sin(\theta_r - 2\pi/3), sin(\theta_r + 2\pi/3)]$

Back emf:
- $e_{abcs} = \omega_r\lambda_m'*[cos(\theta_r), cos(\theta_r - 2\pi/3), cos(\theta_r + 2\pi/3)]$

### q-d Coordinates:

These variables can be transformed into q-d coordinates with the Park's transformation matrix: $K_s^r$
- $f_{qds} = K_s^r(\theta_r)f_{abcs}$

d-axis is aligned with rotor north pole:
- $e_{qs} = \omega_r\lambda_m'$ and $e_{ds} = 0$

State voltage equations:
- $v_{qs} = r_si_{qs} + L_{ss}\frac{di_{qs}}{dt} + \omega_rL_{ss}i_{ds} + e_{qs}$

- $v_{ds} = r_si_{ds} + L_{ss}\frac{di_{ds}}{dt} - \omega_rL_{ss}i_{ds}$

Where: $L_{ss} = L_{ls} + 1.5L_m$

Torque of round rotor machine:
- $T_e = \frac{3P}{4}\lambda_m'i_{qs}$

Mechanical equations:
- $\frac{d\omega_r}{dt} = \frac{P}{2J}(T_e - T_m)$, $T_m$ is mechanical load including friction
- $\frac{d\theta_r}{dt} = \omega_r$

### Modeling VSI:

Switching logic is determined by compensated angular position ($\theta'$).
The compensated angle is the sum of the estimated position from the Hall signals and the firing angle compensation.
- $\theta' = \hat{\theta} + \Delta\phi_v$

The VSI is supplied with a fixed dc source with a voltage of $V_{dc}$, but it is modulated with PWM with a duty cycle, $d_{pwm}$

Estimated position to construct commutation logic is $\theta = \theta_r + \phi_v$.
Hall sensors output 1s or 0s based on the position of rotor magnetic poles. 

The hall state, $h_s$ can be determined through the signals of the other hall effect sensors.

A prediction for angular velocity can be made by seeing how long it takes for the hall state to change. $\hat{\omega_r} = \frac{\Delta\theta}{\Delta t}$
Then the predicted angle can be calculated from integrating predicted angular velocity and adding it to the current hall state angle. From this prediction, rotor angle can be determined by subtracting the advanced firing angle. 

### Average voltage values:

Average values of variables can be found in the motor. The average value depends largely on the advanced firing angle.

- $\bar{v_{qs}^r} = \frac{2}{\pi}V_{dc}cos(\phi_v)$

- $\bar{v_{ds}^r} = -\frac{2}{\pi}V_{dc}sin(\phi_v)$

For 180-deg operation, in COM advanced firing angle fixed to zero.

### MTPV:
To find the desired $\phi_v$ take the partial derivative of torque w.r.t the firing angle to optimize.
This results in the following angle:

$\phi_v = \phi_{MTPV} = \arctan{\frac{\omega_r L_{ss}}{r_s}}$

Controller: Use PI controller to set the speed of the motor. Calculate the firing angle through this equation based on the current speed and adjust.

### MTPA: 
Control $\phi_v$ so that the d-axis current of the stator is maintained at zero. 
Optimizing similarly to the MTPV angle, a complicated expression for the MTPA angle can be determined which is based on rotation speed and the time constant of the stator, and effective DC voltage from PWM. 

Controller: Similarly to MTPV, use a PI controller to control the speed. Then use the current speed and PWM duty cycle to calculate the firing angle. 

### Hybrid strategy:
Use both MTPV and MTPA depending on the operating condition to get the best performance.
For BLDC we want steady state to be controlled by MTPA and transients to be controlled by MTPV. 

How do we know whether we are in a transient or steady state? Find a speed error index which is the normalized difference between actual and desired speed.

$e = \frac{\omega_{ref} - \omega_{r}}{\omega_{ref}}$

Create thresholds for the value of e to determine whether it should be transient behavior or steady state. 

Low-pass filters are added for dc voltage commands and firing angles to avoid torque spikes and smoothen transitions between operating modes. 
$LPF = \frac{1}{\tau_L s + 1}$


# MTPA and MTPV Strategies of BLDCs with 120-Deg Operation

120 degree commutation is a method that achieves an approximation of MTPA. Winding inductance causes fundamental component of the phase current to experience delays which is very noticable at high inductance and small resistance (time constant is L/R). 
- PI controller can regulate average d-axis current to zero to keep MTPA. 
- MTPV strategy is via a 2D LUT. Firing angles are pre-tabulated corresponding to different DC voltages and machine speeds.
    To find the angle take the derivative of the steady-state torque equation w.r.t firing angle and set to zero to optimize for maximum torque for a given DC voltage. This leads to a fourth order function w.r.t sin(theta_v). This can be solved numerically, then tabulated into a LUT. 

Control:
- Effective DC voltage controlled with PWM. PWM-ON mode is used which is using PWM for 60deg, then holding the voltage at constant DC for 60deg. Reduces torque ripple. 

Result:
- MTPV scheme increases torque ripple, but increases voltage-to-torque ration. 

# Extended Conduction of BLDC Motors Under MTPA Control

Researching conduction angles between 120deg and 180deg. Expected to reduce torque ripple, current ripple, and iron losses from PWM. The challenge here is that for 120deg and 180 deg commutation, the commutation points could be determined directly from Hall sensor signals. For angles between, commutation needs to also be determined from the compensated position based on the estimated position. 

The position is estimated based on: $\theta(m) = \theta_H(t_n) + mT_I\hat{\omega_r}$
Where m is the $m_{th}$ sampling point between $t_n$ and $t_{n+1}$. $T_I$ is the fast hardware sampling period. 

Filter estimate speed to reduce affect of unbalanced or misaligned hall sensors. 

How to implemented extended MTPA:
 - Still can use a PI controller to regulate the d-axis current to zero.

### Some thoughts:
For conduction angles between 120deg and 180deg, we need to estimate the current angle based on estimated speed and previous Hall state. What if an analog hall sensor were used instead to have a better estimate of the current position?